In [1]:
pip install opencv-python mediapipe matplotlib

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import cv2
import mediapipe as mp
import numpy as np
from scipy.spatial import distance

# Initialize Mediapipe Pose.
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Initialize Mediapipe Hands.
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()

# Initialize Mediapipe Drawing.
mp_drawing = mp.solutions.drawing_utils

# Open a video capture object.
cap = cv2.VideoCapture(0)

# Create a canvas for drawing.
canvas = None

# Variables to store the previous positions of the index fingers.
prev_left_index = None
prev_right_index = None

# Variables to store the drawing state and trajectory.
drawing = False
trajectory = []

# Modes
MODE_DRAW = 1
MODE_ERASE = 2
MODE_COLOR = 3
mode = MODE_DRAW

# Default color
color = (0, 255, 0)

# Color options
colors = [(0, 255, 0), (0, 0, 255), (255, 0, 0), (0, 255, 255), (255, 255, 0), (255, 0, 255)]

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Initialize the canvas.
    if canvas is None:
        canvas = np.zeros_like(frame)
    
    # Convert the BGR image to RGB.
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Process the image and detect the pose.
    pose_result = pose.process(rgb_frame)
    
    # Process the image and detect hands.
    hand_result = hands.process(rgb_frame)
    
    # Draw the pose annotation on the image.
    annotated_image = frame.copy()
    
    if pose_result.pose_landmarks:
        mp_drawing.draw_landmarks(
            annotated_image, pose_result.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        
    if hand_result.multi_hand_landmarks:
        for hand_landmarks in hand_result.multi_hand_landmarks:
            mp_drawing.draw_landmarks(annotated_image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            
            # Get the index finger tip, thumb tip and middle finger tip coordinates.
            index_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
            thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
            middle_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]
            h, w, c = frame.shape
            index_finger_pos = (int(index_finger_tip.x * w), int(index_finger_tip.y * h))
            thumb_pos = (int(thumb_tip.x * w), int(thumb_tip.y * h))
            middle_finger_pos = (int(middle_finger_tip.x * w), int(middle_finger_tip.y * h))
            
            # Determine if the hand is left or right.
            hand_label = hand_result.multi_handedness[hand_result.multi_hand_landmarks.index(hand_landmarks)].classification[0].label
            
            # Check if the thumb and index finger are close to each other to switch modes.
            distance_thumb_index = distance.euclidean(index_finger_pos, thumb_pos)
            mode_threshold = 40  # Adjust this threshold as needed
            
            if distance_thumb_index < mode_threshold:
                if mode == MODE_DRAW:
                    mode = MODE_ERASE
                elif mode == MODE_ERASE:
                    mode = MODE_COLOR
                else:
                    mode = MODE_DRAW
            
            # Check if the index and middle fingers are close together to enable drawing/erasing.
            distance_between_fingers = distance.euclidean(index_finger_pos, middle_finger_pos)
            eraser_threshold = 40  # Adjust this threshold as needed
            
            if mode == MODE_DRAW:
                # Start drawing when index and middle fingers are not close.
                if distance_between_fingers >= eraser_threshold:
                    drawing = True
                    trajectory.append(index_finger_pos)
                    if len(trajectory) > 1:
                        cv2.line(canvas, trajectory[-2], trajectory[-1], color, 5)
                else:
                    drawing = False
                    trajectory = []
            
            elif mode == MODE_ERASE:
                # Erase lines when index and middle fingers are close to each other.
                if distance_between_fingers < eraser_threshold:
                    eraser_size = 50
                    cv2.circle(canvas, index_finger_pos, eraser_size, (0, 0, 0), -1)
            
            elif mode == MODE_COLOR:
                # Switch color when in color mode.
                for i, col in enumerate(colors):
                    color_pos = (50 + i * 60, 50)
                    cv2.circle(annotated_image, color_pos, 20, col, -1)
                    if distance.euclidean(index_finger_pos, color_pos) < 20:
                        color = col

    # Combine the canvas and the annotated image.
    annotated_image = cv2.addWeighted(annotated_image, 1, canvas, 0.5, 0)
    
    # Display the annotated image.
    cv2.imshow('Body and Hand Drawing with Shape Detection and Eraser', annotated_image)
    
    # Break the loop on 'q' key press.
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

# Release resources.
cap.release()
cv2.destroyAllWindows()

C:\Users\jaya verma\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
